In [5]:
import os

HOME = os.getcwd()
print(HOME)

%cd {HOME}
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd {HOME}/GroundingDINO
!pip install -q -e .

CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
print(CONFIG_PATH, "; exist:", os.path.isfile(CONFIG_PATH))

/Users/jc/code/aipi540-module1/notebooks/dino
/Users/jc/code/aipi540-module1/notebooks/dino
fatal: destination path 'GroundingDINO' already exists and is not an empty directory.
/Users/jc/code/aipi540-module1/notebooks/dino/GroundingDINO
/Users/jc/code/aipi540-module1/notebooks/dino/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py ; exist: True


In [7]:
%cd {HOME}
!mkdir {HOME}/weights
%cd {HOME}/weights

!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

WEIGHTS_NAME = "groundingdino_swint_ogc.pth"
WEIGHTS_PATH = os.path.join(HOME, "weights", WEIGHTS_NAME)
print(WEIGHTS_PATH, "; exist:", os.path.isfile(WEIGHTS_PATH))

/Users/jc/code/aipi540-module1/notebooks/dino
mkdir: /Users/jc/code/aipi540-module1/notebooks/dino/weights: File exists
/Users/jc/code/aipi540-module1/notebooks/dino/weights
/Users/jc/code/aipi540-module1/notebooks/dino/weights/groundingdino_swint_ogc.pth ; exist: True


In [10]:
%cd {HOME}/GroundingDINO

from groundingdino.util.inference import load_model, load_image, predict, annotate
from groundingdino.util.inference import Model

model = load_model(CONFIG_PATH, WEIGHTS_PATH)

GROUNDING_DINO_CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
print(GROUNDING_DINO_CONFIG_PATH, "; exist:", os.path.isfile(GROUNDING_DINO_CONFIG_PATH))

GROUNDING_DINO_CHECKPOINT_PATH = os.path.join(HOME, "weights", WEIGHTS_NAME)
print(GROUNDING_DINO_CHECKPOINT_PATH, "; exist:", os.path.isfile(GROUNDING_DINO_CHECKPOINT_PATH))

grounding_dino_model = Model(model_config_path=GROUNDING_DINO_CONFIG_PATH, model_checkpoint_path=GROUNDING_DINO_CHECKPOINT_PATH)

/Users/jc/code/aipi540-module1/notebooks/dino/GroundingDINO
final text_encoder_type: bert-base-uncased
/Users/jc/code/aipi540-module1/notebooks/dino/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py ; exist: True
/Users/jc/code/aipi540-module1/notebooks/dino/weights/groundingdino_swint_ogc.pth ; exist: True
final text_encoder_type: bert-base-uncased


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
import torch
from torchvision.ops import box_convert
import numpy as np
from PIL import Image
import supervision as sv

In [3]:
TEXT_PROMPT = "snake"
BOX_TRESHOLD = 0.25
TEXT_TRESHOLD = 0.25

def get_bounding_box(path):
  image_source, image = load_image(path)

  boxes, logits, phrases = predict(
      model=model,
      image=image,
      caption=TEXT_PROMPT,
      box_threshold=BOX_TRESHOLD,
      text_threshold=TEXT_TRESHOLD
  )

  annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=phrases)

  %matplotlib inline
  sv.plot_image(annotated_frame, (16, 16))

  return boxes[0].numpy()


In [4]:
img_path = "../../data/processed/train/agkistrodon_contortrix-912622/10501066.jpg"
box = get_bounding_box(img_path)
box

NameError: name 'load_image' is not defined

In [2]:
import cv2

center_x_pct = box[0]
center_y_pct = box[1]
width_pct = box[2]
height_pct = box[3]

img = cv2.imread(img_path)

img_height, img_width, _ = img.shape

# Convert normalized coordinates to pixel values
center_x = int(center_x_pct * img_width)
center_y = int(center_y_pct * img_height)
width = int(width_pct * img_width)
height = int(height_pct * img_height)

print("Center:", (center_x, center_y))
print("Width:", width)
print("Height:", height)

x1 = center_x - width // 2
x2 = center_x + width // 2
y1 = center_y - height // 2
y2 = center_y + height // 2

print("Cropping image from", (x1, y1), "to", (x2, y2))

cropped_img = img[y1:y2, x1:x2]

cv2_imshow(cropped_img)

NameError: name 'box' is not defined